<a href="https://colab.research.google.com/github/ShaunZhxiong/musa-611-midterm/blob/main/DogNYC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dogNYC

In [2]:
% pip install geopandas &> /dev/null
% pip install cenpy &> /dev/null
import cenpy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Dog Runs

In [3]:
dogRun = gpd.read_file("/content/NYC Parks Dog Runs.geojson")
dogRun.head(n=2)

,zipcode,name,system,surface,dog_area_t,featuresta,parentid,gispropnum,objectid,communityb,department,precinct,seating,councildis,borough,geometry
0,10038,Fishbridge Garden Dog Run,M291-DOGAREA0043,None,Dog Run,Active,M291,M291,292,101,M-01,1,None,1,M,"MULTIPOLYGON (((-74.00165 40.70933, -74.00099 ..."
1,11360,Little Bay Park Dog Run,Q010A-DOGAREA0003,None,Dog Run,Active,Q010A-ZN01,Q010A,318,407,Q-11,109,None,19,Q,"MULTIPOLYGON (((-73.79285 40.78794, -73.79280 ..."


# Pet License

To aggregate data to geolation, i used zcta5 to do so

In [4]:
dogLicenses = pd.read_csv("/content/NYC_Dog_Licensing_Dataset.csv")
dogLicenses

,RowNumber,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year,Unique Dog ID
0,1,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,NaN,10035,09/12/2014,09/12/2017,2016.0,NaN
1,2,YOGI,M,2010,Boxer,NaN,10465,09/12/2014,10/02/2017,2016.0,NaN
2,3,ALI,M,2014,Basenji,NaN,10013,09/12/2014,09/12/2019,2016.0,NaN
3,4,QUEEN,F,2013,Akita Crossbreed,NaN,10013,09/12/2014,09/12/2017,2016.0,NaN
4,5,LOLA,F,2009,Maltese,NaN,10028,09/12/2014,10/09/2017,2016.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
493067,147341,UNKNOWN,F,2020,Poodle Crossbreed,NaN,10025,03/17/2021,03/17/2022,NaN,NaN
493068,147342,UNKNOWN,F,2020,"Coonhound, Treeing Walker",NaN,11201,03/17/2021,03/17/2022,NaN,NaN
493069,147343,UNKNOWN,M,2020,Dachshund,NaN,11206,03/17/2021,03/17/2022,NaN,NaN
493070,147344,UNKNOWN,M,2020,Goldendoodle,NaN,10019,03/17/2021,03/17/2022,NaN,NaN


In [17]:
licenseZip = pd.DataFrame(dogLicenses["ZipCode"].unique())
allZipGeo = pd.DataFrame()

acs = cenpy.remote.APIConnection("ACSDT5Y2019")
acs.set_mapservice("tigerWMS_ACS2019")
allZipGeo = pd.DataFrame()
for i in range(len(licenseZip)):
  try:
    where_clause = f"GEOID={licenseZip[0][i]}"
    censusTract = acs.mapservice.layers[2].query(where=where_clause)
    print("Process: "+str(i)) if i % 100 == 0 else 0
    allZipGeo = pd.concat([allZipGeo,censusTract[["GEOID","geometry"]]])
  except:
    pass

Process: 0
Process: 100
Process: 200
Process: 300
Process: 400
Process: 500
Process: 600


In [19]:
allZipGeo['GEOID'] = allZipGeo['GEOID'].astype(int)

In [26]:
allZipGeoToGo = allZipGeo.to_crs(epsg=4326)

In [30]:
dogLicensesToGo = pd.merge(allZipGeoToGo,dogLicenses,left_on="GEOID",right_on="ZipCode",how="inner")

In [35]:
dogLicensesToGo.head(n=5)

,GEOID,geometry,RowNumber,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year,Unique Dog ID
0,10035,"MULTIPOLYGON (((-73.93624 40.78569, -73.93551 ...",1,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,NaN,10035,09/12/2014,09/12/2017,2016.0,NaN
1,10035,"MULTIPOLYGON (((-73.93624 40.78569, -73.93551 ...",82,BRODY,M,2006,Cocker Spaniel,NaN,10035,09/16/2014,09/16/2016,2016.0,NaN
2,10035,"MULTIPOLYGON (((-73.93624 40.78569, -73.93551 ...",199,ROCKO,M,2013,Chihuahua,NaN,10035,09/27/2014,09/27/2017,2016.0,NaN
3,10035,"MULTIPOLYGON (((-73.93624 40.78569, -73.93551 ...",282,SENSATION,F,2014,Mastiff,NaN,10035,10/05/2014,10/05/2016,2016.0,NaN
4,10035,"MULTIPOLYGON (((-73.93624 40.78569, -73.93551 ...",290,TEMPTATION,M,2014,Mastiff,NaN,10035,10/05/2014,10/05/2016,2016.0,NaN


# Pet Store

In [24]:
import requests
import json
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(node["shop"="pet"](40.492920,-74.262393,40.913656220576904,-73.69789062134433););
out center;
"""
response = requests.get(overpass_url,params={'data': overpass_query})
petStore = response.json()

In [38]:
petStore['elements'][0]

{'id': 1157647167,
 'lat': 40.6774406,
 'lon': -73.9691709,
 'tags': {'addr:housenumber': '628',
  'addr:street': 'Vanderbilt Avenue',
  'delivery': 'yes',
  'name': 'Acme Pet Food',
  'opening_hours': 'Mo-Fr 10:00-19:00; Sa 10:00-18:00; Su 11:00-17:00',
  'phone': '+1-718-789-8062',
  'shop': 'pet'},
 'type': 'node'}

In [26]:
with open('/content/petStore.json', 'w') as f:
    json.dump(petStore, f)